In [ ]:
# YOLOv8 Training Script

from ultralytics import YOLO
import os

# Load a pretrained YOLOv8n variant (custom lightweight model)
model = YOLO('../src/human_detector/models/yolov11n.pt')


# Training with simple parameters
#    - data: path to your data.yaml
#    - epochs, imgsz, project, name: same CLI args
# results = model.train(
#     data    = 'data.yaml',
#     epochs  = 1,
#     imgsz   = 640,
#     project = 'human_robot',
#     name    = 'exp1'
# )

# Train the model with custom augmentation settings and longer epochs
results = model.train(
    data='data.yaml',               # Dataset config file
    epochs=120,                     # Train longer
    imgsz=416,                      # Moderate image size
    batch=2,                        # Small batch for limited VRAM
    device='0',                     # Use GPU 0
    half=True,                      # Mixed precision for speed + memory
    workers=2,                      # Data loader threads
    cache=False,                    # Avoid caching to save memory
    patience=20,                    # Early stop if mAP stops improving
    hyp='custom_hyp.yaml',          # Use low-light augmentation settings
    project='human_robot',
    name='exp_lowlight'
)

# Save trained model path
save_dir = str(results.save_dir)
best_pt = os.path.join(save_dir, 'weights', 'best.pt')
print(f"✅  Trained checkpoint saved to: {best_pt}")


# 🧠 YOLOv8 Training Metrics Cheatsheet

## 🔢 Loss Curves

| Metric            | Meaning                                                | Goal / Good Trend                |
|-------------------|--------------------------------------------------------|----------------------------------|
| `train/box_loss`  | Bounding box regression loss (training)                | ↓ Steady decline                 |
| `train/cls_loss`  | Classification loss (training)                         | ↓ Should fall and stabilize     |
| `train/dfl_loss`  | Distribution Focal Loss (for bounding box quality)     | ↓ Smooth drop                   |
| `val/box_loss`    | Box loss on validation set                             | ↓ Best indicator of generalization |
| `val/cls_loss`    | Classification loss on validation                      | ↓ Falling → Good model fit      |
| `val/dfl_loss`    | DFL loss on validation                                 | ↓ Matches train trend = no overfit |

---

## 📈 Detection Metrics

| Metric                | Description                                                 | Goal / Interpretation           |
|------------------------|-------------------------------------------------------------|----------------------------------|
| `metrics/precision(B)` | % of correct detections out of all predictions              | ↑ High = few false positives     |
| `metrics/recall(B)`    | % of actual objects detected                                | ↑ High = few missed detections   |
| `metrics/mAP50(B)`     | Accuracy of boxes at 50% IoU threshold                     | ↑ Main object detection metric   |
| `metrics/mAP50-95(B)`  | Average mAP over IoU 0.5 to 0.95 (stricter)                | ↑ Robust performance measure     |

---

## 🧪 Ideal Curve Patterns

- **Losses**: should steadily decrease.
- **Validation loss** rising while training loss decreases → possible **overfitting**.
- **Precision & Recall**: should increase and plateau near 1.0.
- **mAP**: aim for **>0.90 mAP50** and **>0.80 mAP50-95** for high-quality results.

---

## ⚠️ Tips

- 🧊 Use `half=True` for mixed precision training on supported GPUs.
- 🧠 Watch `val/box_loss` and `mAP50-95` to catch overfitting early.
- 🧪 If metrics plateau early, try increasing `epochs`, `imgsz`, or improving labels.
- 💾 Check `runs/train/.../results.png` for curve visualization.

---
